In [5]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from copy import deepcopy
from pybedtools import BedTool

In [6]:
def find_asnr(seq, match_length, max_mismatch, strand):
    indices = []
    i = 0
    length = len(seq)
    if strand == "+":
        while i < length:
            if seq[i] != 'A':
                i += 1
                continue
            j = i + 1
            non_A_count = 0
            while j < len(seq) and non_A_count <= max_mismatch:
                if seq[j] != 'A':
                    non_A_count += 1
                j += 1
            while non_A_count > max_mismatch: 
                j -= 1
                if seq[j] != 'A':
                    non_A_count -= 1
            if j - i >= match_length:
                indices.append(i)
            i = j
    else:
        seq = seq[::-1]
        while i < length:
            if seq[i] != 'T':
                i += 1
                continue
            j = i + 1
            non_T_count = 0
            while j < len(seq) and non_T_count <= max_mismatch:
                if seq[j] != 'T':
                    non_T_count += 1
                j += 1
            while non_T_count > max_mismatch: 
                j -= 1
                if seq[j] != 'T':
                    non_T_count -= 1
            if j - i >= match_length:
                indices.append(length - i - 1)
            i = j
    return indices

def search_origin_coord(
    orig_coord,
    search_start,
    search_end,
):
    accumulated_intervals = [[0, end-start] for start, end in orig_coord]
    for i in range(1, len(accumulated_intervals)):
        accumulated_intervals[i][0] = accumulated_intervals[i-1][1]
        accumulated_intervals[i][1] += accumulated_intervals[i][0]
    accumulated_intervals = [x[1] for x in accumulated_intervals]
    accumulated_intervals = np.array([0] + accumulated_intervals)
    start_idx = np.searchsorted(accumulated_intervals, search_start) - 1
    start_idx = max(0, start_idx)
    end_idx = np.searchsorted(accumulated_intervals, search_end) - 1
    return_orig_coord = deepcopy(orig_coord[start_idx:end_idx+1])
    if len(return_orig_coord) == 0:
        return_orig_coord = None
    else:
        return_orig_coord[0][0] = return_orig_coord[0][0] - accumulated_intervals[start_idx] + search_start
        return_orig_coord[-1][1] = return_orig_coord[-1][1] - accumulated_intervals[end_idx] + search_end
    return return_orig_coord


In [7]:
gtf_path = "/root/nfsdata/REFERENCE/GENOME/MOUSE/vm25/gencode.vM25.annotation.gtf"
genome_path = "/root/nfsdata/REFERENCE/GENOME/MOUSE/vm25/GRCm38.p6.genome.fa"

In [17]:
upstream_extend_length = 200
downstream_extend_length = 50

In [9]:
gtf = pd.read_csv(gtf_path, comment='#', sep='\t', header=None, 
                      names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
genome_dict = SeqIO.to_dict(list(SeqIO.parse(genome_path, "fasta")))

In [19]:
exon_gtf = gtf.loc[(gtf['feature'] == 'exon') & (gtf["seqname"] != "chrM"), :].sort_values(by=['seqname', 'start']).copy()
exon_gtf["seq"] = exon_gtf.apply(lambda x: str(genome_dict[x['seqname']].seq[x['start']-1:x['end']]) , axis=1)
exon_gtf['transcript_id'] = exon_gtf['attribute'].apply(lambda x: x.split(";")[1].split(" ")[2].replace('"', ''))

In [20]:
transcript_seq = pd.DataFrame(exon_gtf.groupby('transcript_id')["seq"].apply("".join))
transcript_seq["strand"] = exon_gtf.groupby('transcript_id')["strand"].first()
transcript_seq["chr"] = exon_gtf.groupby('transcript_id')["seqname"].first()
transcript_seq["orig_coord"] = exon_gtf.groupby('transcript_id')[['start', 'end']].apply(lambda x: [[start, end] for start, end in zip(x['start'], x['end'])])
transcript_seq["asnr_stop"] = transcript_seq.apply(lambda x: find_asnr(x["seq"], 10, 1, x["strand"]), axis=1)

In [21]:
transcript_seq["asnr_expand"] = transcript_seq.apply(lambda x: [[max(0, i-upstream_extend_length), i+downstream_extend_length] for i in x["asnr_stop"]] if x["strand"] == "+" else [[i-downstream_extend_length, min(i+upstream_extend_length, len(x["seq"]))] for i in x["asnr_stop"]], axis=1)
transcript_seq["asnr_coord"] = transcript_seq.apply(lambda x: [search_origin_coord(x["orig_coord"], start, end) for start, end in x["asnr_expand"]], axis=1)

In [22]:
exonic_asnr_df = pd.merge(transcript_seq["asnr_coord"].explode().explode().dropna().reset_index(), transcript_seq["chr"].reset_index())
exonic_asnr_df = pd.merge(exonic_asnr_df, transcript_seq["strand"].reset_index())
exonic_asnr_df["start"] = exonic_asnr_df["asnr_coord"].apply(lambda x: x[0])
exonic_asnr_df["end"] = exonic_asnr_df["asnr_coord"].apply(lambda x: x[1])
exonic_asnr_df["name"] = exonic_asnr_df["transcript_id"]
exonic_asnr_df["score"] = "."

In [23]:
gene_bed = BedTool.from_dataframe(gtf.loc[(gtf['feature'] == 'gene') & (gtf["seqname"] != "chrM"), ['seqname', 'start', 'end', 'feature','score','strand']])
exon_bed = BedTool.from_dataframe(exon_gtf[['seqname', 'start', 'end', 'feature','score','strand']])
intron_bed = gene_bed.subtract(exon_bed, s=True).sort()
intron_df = intron_bed.to_dataframe()
intron_df["seq"] = intron_df.apply(lambda x: str(genome_dict[x['chrom']].seq[x['start']:x['end']]) , axis=1)
intron_df["asnr_stop"] = intron_df.apply(lambda x: find_asnr(x["seq"], 10, 1, x["strand"]), axis=1)
intron_df["asnr_expand"] = intron_df.apply(lambda x: [[max(0, i-upstream_extend_length), i+downstream_extend_length] for i in x["asnr_stop"]] if x["strand"] == "+" else [[i-downstream_extend_length, min(i+upstream_extend_length, len(x["seq"]))] for i in x["asnr_stop"]], axis=1)
intron_df["asnr_coord"] = intron_df.apply(lambda x: [[x["start"]+start, x["start"]+end] for start, end in x["asnr_expand"]], axis=1)

In [24]:
intronic_asnr_df = pd.merge(intron_df["asnr_coord"].explode().dropna().reset_index(), intron_df[['chrom', 'strand']].reset_index())
intronic_asnr_df["start"] = intronic_asnr_df["asnr_coord"].apply(lambda x: x[0])
intronic_asnr_df["end"] = intronic_asnr_df["asnr_coord"].apply(lambda x: x[1])
intronic_asnr_df["name"] = "intron"
intronic_asnr_df["score"] = "."
intronic_asnr_df["chr"] = intronic_asnr_df["chrom"]

In [25]:
exonic_asnr_df.loc[:, ["chr", "start", "end", "name", "score", "strand"]].to_csv("./mm10_exon_asnr.bed", sep='\t', header=False, index=False)
intronic_asnr_df.loc[:, ["chr", "start", "end", "name", "score", "strand"]].to_csv("./mm10_intron_asnr.bed", sep='\t', header=False, index=False)

In [ ]:
# bedtools subtract -A -s -a mm10_exon_asnr.bed -b /root/apabenchmark/data/annotation/mouse_integrated_pas.bed > mm10_exon_asnr_filtered.bed
# bedtools subtract -A -s -a mm10_intron_asnr.bed -b /root/apabenchmark/data/annotation/mouse_integrated_pas.bed > mm10_intron_asnr_filtered.bed